# Лабораторная работа №6

In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет со свойствами различных грибов, целевым параметром является съедобность или несъедобность гриба.

In [4]:
mushrooom_dataset = pd.read_csv('MushroomDataset/secondary_data.csv',delimiter=';')

Выделяем таргетный столбец в отдельный Series, а все остальные столбы в DataFrame. Также удаляем все столбцы, где большая часть данных типа NaN и заменяем все классы на номер буквы в алфавите.

In [5]:
y = mushrooom_dataset['class']
X = mushrooom_dataset.drop('class',axis=1)
X = X.drop(columns=['cap-surface','gill-spacing','gill-attachment','stem-root',
                    'stem-surface','veil-type','veil-color','ring-type','spore-print-color'])
X.replace({'a':1,'b':2,'c':3,'d':4,'e':5,'f':6,'g':7,
           'h':8,'i':9,'j':10,'k':11,'l':12,'m':13,
           'n':14,'o':15,'p':16,'q':17,'r':18,'s':19,
           't':20,'u':21,'v':22,'w':23,'x':24,'y':25,'z':26},inplace=True)

После устранения пропусков нормализуем все значения.

In [6]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем данные на тренировочную и тестовую части.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.2)

Найдём лучшие гиперпараметры, для этого используем GridSearchCV, куда передадим диапазон возможных значений. Будем искать оптимальное значение гиперпараметров iterations, learning_rate и depth, которые отвечает за кол-во итераций, скорость обучения и максимальная глубина.

In [9]:
cat_boost_classifier = CatBoostClassifier()
cat_boost_params ={
    "iterations": range(1,6),
    'learning_rate': [0.1,0.25,0.5,0.75,1],
    'depth':range(1,6),
}
cat_boost_grid = GridSearchCV(cat_boost_classifier, cat_boost_params, cv=2, n_jobs=1,verbose=3)
cat_boost_grid.fit(X_train, y_train)
best_cat_boost_params = cat_boost_grid.best_params_

Fitting 2 folds for each of 125 candidates, totalling 250 fits
0:	learn: 0.6866771	total: 139ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.1;, score=0.610 total time=   0.2s
0:	learn: 0.6868001	total: 1.78ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.1;, score=0.609 total time=   0.1s
0:	learn: 0.6782243	total: 1.95ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.25;, score=0.610 total time=   0.1s
0:	learn: 0.6785072	total: 1.68ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.25;, score=0.609 total time=   0.1s
0:	learn: 0.6675293	total: 1.86ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.5;, score=0.610 total time=   0.1s
0:	learn: 0.6680112	total: 1.68ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.5;, score=0.609 total time=   0.1s
0:	learn: 0.6609861	total: 1.88ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.75;, score=0.610 to

После того, как были найдены оптимальные гиперпараметры, можно вывести различные метрики, такие как точность, полнота, f-мера и поддержка.

In [10]:
best_des_tree_model = CatBoostClassifier()
best_des_tree_model.fit(X_train, y_train)
des_tree_predicted = best_des_tree_model.predict(X_test)
#print('Used params:',best_des_tree_params)
print('Evaluation:\n', metrics.classification_report(y_test, des_tree_predicted,digits=5))

Learning rate set to 0.054212
0:	learn: 0.6748734	total: 14.7ms	remaining: 14.7s
1:	learn: 0.6618039	total: 27.5ms	remaining: 13.7s
2:	learn: 0.6461327	total: 41.1ms	remaining: 13.7s
3:	learn: 0.6325608	total: 52.7ms	remaining: 13.1s
4:	learn: 0.6182258	total: 64.7ms	remaining: 12.9s
5:	learn: 0.6033242	total: 79.5ms	remaining: 13.2s
6:	learn: 0.5942961	total: 92.7ms	remaining: 13.1s
7:	learn: 0.5811940	total: 106ms	remaining: 13.2s
8:	learn: 0.5680195	total: 119ms	remaining: 13.2s
9:	learn: 0.5597398	total: 132ms	remaining: 13.1s
10:	learn: 0.5505980	total: 145ms	remaining: 13s
11:	learn: 0.5399876	total: 157ms	remaining: 12.9s
12:	learn: 0.5309665	total: 170ms	remaining: 12.9s
13:	learn: 0.5246761	total: 183ms	remaining: 12.9s
14:	learn: 0.5190278	total: 199ms	remaining: 13.1s
15:	learn: 0.5132273	total: 213ms	remaining: 13.1s
16:	learn: 0.5056131	total: 227ms	remaining: 13.1s
17:	learn: 0.4986433	total: 241ms	remaining: 13.1s
18:	learn: 0.4939990	total: 254ms	remaining: 13.1s
19:	le